# Screener 02

https://readmedium.com/en/https:/wire.insiderfinance.io/identifying-breakout-stocks-with-python-a-data-driven-selection-guide-d8a3d1ba172e
https://github.com/shashankvemuri/Finance/blob/master/find_stocks/get_rsi_tickers.py

https://medium.com/@redeaddiscolll
https://medium.com/@py.chin315
https://github.com/peiyingchin/Medium/blob/main/Trading%20strategy%20with%2055%25%20win%20chance/Trading%20strategy%20with%2055%25%20win%20chance.ipynb

In [6]:
import numpy as np
import numba as nb
import yfinance as yf

@nb.jit(nopython = True)
def explicit_heat_smooth(prices: np.array,
                         t_end: float = 5.0) -> np.array:
    '''
    Smoothen out a time series using a explicit finite difference method.
    Parameters
    ----------
    prices : np.array
        The price to smoothen
    t_end : float
        The time at which to terminate the smootheing (i.e. t = 2)
    Returns
    -------
    P : np.array
        The smoothened time-series
    '''
    
    k = 0.1 # Time spacing, must be < 1 for numerical stability
    
    # Set up the initial condition
    P = prices
    
    t = 0
    while t < t_end:
        # Solve the finite difference scheme for the next time-step
        P = k*(P[2:] + P[:-2]) + P[1:-1]*(1-2*k)
        
        # Add the fixed boundary conditions since the above solves the interior
        # points only
        P = np.hstack((
            np.array([prices[0]]),
            P,
            np.array([prices[-1]]),
        ))
        t += k

    return P

        
@nb.jit(nopython = True)
def check_consolidation(prices: np.array,
                        perc_change_days: int,
                        perc_change_thresh: float,
                        check_days: int) -> int:
    '''
    Smoothen the time-series and check for consolidation, see the
    docstring of find_consolidation for the parameters
    '''
    
    # Find the smoothed representation of the time series
    prices = explicit_heat_smooth(prices)
    
    # Perc change of the smoothed time series to perc_change_days days prior
    perc_change = prices[perc_change_days:]/prices[:-perc_change_days] - 1
    
    consolidating = np.where(np.abs(perc_change) < perc_change_thresh, 1, 0)
    
    # Provided one entry in the last n days passes the consolidation check,
    # we say that the financial instrument is in consolidation on the end day
    if np.sum(consolidating[-check_days:]) > 0:
        return 1
    else:
        return 0
    
    
@nb.jit(nopython = True)
def find_consolidation(prices: np.array,
                       days_to_smooth: int = 50,
                       perc_change_days: int = 5,
                       perc_change_thresh: float = 0.015,
                       check_days: int = 5) -> np.array:
    '''
    Return a binary array to indicate whether each of the data-points are
    classed as consolidating or not
    Parameters
    ----------
    prices : np.array
        The price time series to check for consolidation
    days_to_smooth : int, optional
        The length of the time-series to smoothen (days). The default is 50.
    perc_change_days : int, optional
        The days back to % change compare against (days). The default is 5.
    perc_change_thresh : float, optional
        The range trading % criteria for consolidation. The default is 0.015.
    check_days : int, optional
        This says the number of lookback days to check for any consolidation.
        If any days in check_days back is consolidating, then the last data
        point is said to be consolidating. The default is 5.
    Returns
    -------
    res : np.array
        The binary array indicating consolidation (1) or not (0)
    '''
    
    res = np.full(prices.shape, np.nan)
    
    for idx in range(days_to_smooth, prices.shape[0]):
        res[idx] = check_consolidation(
            prices = prices[idx-days_to_smooth:idx],
            perc_change_days = perc_change_days,
            perc_change_thresh = perc_change_thresh,
            check_days = check_days,
        )
        
    return res
    

if __name__ == '__main__':
    
    df = yf.download('TSLA').reset_index()
    df.loc[:, 'consolidating'] = find_consolidation(df['Close'].values)
    df.dropna()

[*********************100%%**********************]  1 of 1 completed


In [ ]:
df = yf.download('TSLA').reset_index()
df.loc[:, 'consolidating'] = find_consolidation(df['Close'].values)
df.loc[:, 'trend_filter'] = trend_filter(df['Close'])
df.loc[:, 'filtered'] = np.where(
    df['consolidating'] + df['trend_filter'] == 2,
    True,
    False,
)


## RSI

In [5]:
# Import Dependencies
import datetime
from pandas_datareader import data as pdr
import sys
import os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
import ta_functions as ta
import tickers as ti

# Get dates for the past year
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

# Load list of S&P 500 tickers from tickers module
tickers = ti.tickers_sp500()

# Initialize lists for overbought and oversold tickers
oversold_tickers = []
overbought_tickers = []

# Retrieve adjusted close prices for the tickers
sp500_data = pdr.get_data_yahoo(tickers, start_date, end_date)['Adj Close']

# Analyze each ticker for RSI
for ticker in tickers:
    try:
        # Create a new DataFrame for the ticker
        data = sp500_data[[ticker]].copy()

        # Calculate the RSI for the ticker
        data["rsi"] = ta.RSI(data[ticker], timeperiod=14)

        # Calculate the mean of the last 14 RSI values
        mean_rsi = data["rsi"].tail(14).mean()

        # Print the RSI value
        print(f'{ticker} has an RSI value of {round(mean_rsi, 2)}')

        # Classify the ticker based on its RSI value
        if mean_rsi <= 30:
            oversold_tickers.append(ticker)
        elif mean_rsi >= 70:
            overbought_tickers.append(ticker)

    except Exception as e:
        print(f'Error processing {ticker}: {e}')

# Output the lists of oversold and overbought tickers
print(f'Oversold tickers: {oversold_tickers}')
print(f'Overbought tickers: {overbought_tickers}')

ModuleNotFoundError: No module named 'ta_functions'